In [38]:
import requests
import time
import json
import gzip
import csv
import codecs
from bs4 import BeautifulSoup
import io
import sys
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer as ps

stemmer = ps()

In [39]:
domain = 'nba.com/article/*'
index = "2019-13"

#domain = 'espn.com/mens-college-basketball/insider/*'
#domain="si.com/college-basketball/*"
#id = article-body, <p>, longform-body

In [42]:
urls = []
cc_url  = "http://index.commoncrawl.org/CC-MAIN-"+index+"-index?"+"url="+domain+"&output=json"
response = requests.get(cc_url)
if response.status_code == 200:
    records = response.content.splitlines()
    for record in records:
        urls.append(json.loads(record))

In [43]:
stop_words = set(stopwords.words('english'))

def getHeader(start, length):
    return {'Range': 'bytes='+str(start)+'-'+str(start+length-1)}

def stemAndRemoveStop(paras):
    res = []
    for para in paras:
        tempRes = []
        for w in para.split(' '):
            if w not in stop_words:
                tempRes.append(stemmer.stem(w))
        res.append(' '.join(tempRes))
    return res

#https://www.bellingcat.com/resources/2015/08/13/using-python-to-mine-common-crawl/
def getContent(record):
    url = 'https://commoncrawl.s3.amazonaws.com/'
    resp = requests.get(url + record['filename'], headers=getHeader(int(record['offset']), int(record['length'])))
    raw_data = io.BytesIO(resp.content)
    f = gzip.GzipFile(fileobj=raw_data)
    data = f.read()
    response = ""
    if len(data):
        try:
            warc, header, response = data.decode(encoding='utf-8', errors='strict').strip().split('\r\n\r\n', 2)
        except:
            pass
    collection=[]
    if response:
        bs = BeautifulSoup(response)
        paras= bs.find_all("p")
        return ' '.join(stemAndRemoveStop(list(map(lambda c: c.text, paras))))
    return response

In [53]:
articles = []
fileCount = 0
for i, webpage in enumerate(urls):
    content = getContent(webpage)
    if content:
        fileCount+=1
        articles.append(getContent(webpage))
    if fileCount==1000:
        break

In [54]:
articles

['john schuhmann nba.com @johnschuhmannarch oct 8, 2016 9:09 PM ET \n            detroit expect miss servic point guard reggi jackson least 12 games.\n           the injuri bug offseason, hit detroit piston play first preseason game thursday. earlier week, piston point guard diagnos tendin left knee. and saturday, head coach stan van gundi said jackson get platelet-rich plasma inject monday. jackson could miss 6-8 weeks, mean piston would without least 12 game much quarter season. rod beard the\xa0detroit news\xa0ha story... jackson previous plasma inject treatment play oklahoma citi thunder 2011. It longest recoveri time options, highest probabl reliev pain extend period. "our hope normal tendin stuff could work appear be, top that, bone bruise," van gundi said. "he pretti signific setback compar guy â€“ marcu (morris) littl bit, ish (smith) littl bit. "it\' pretti typic stuff that\' hope reggie, way turn out." jackson\' absenc big setback team tri take anoth step forward compet decen

In [55]:
for i, article in enumerate(articles):
    if article:
        file = open('ccNBAarticles/Article'+str(i)+'.txt', 'w')
        file.write(article)
        file.close()